# Freitas Plume Rise with McBEF Inputs

This notebook used the FreitasPlume extension to compute theh vertical mass distribution based on
smoldering and flaming heatfluxes retrieved by the McBEF algorithm.


In [1]:
import os

from IPython.display import display, Markdown

import numpy as np
import xarray as xr

from plumerise import bioma   as bm
from plumerise import freitas as pr

import matplotlib.pyplot as plt
%matplotlib inline  


## McBEF Level 2 Data and Sampled Meteorology

In [2]:
def getFiresMetOneDay(day,Verbose=False):
    """
    Ingests McBEF retrievals and sampled meteorology for one day.
    """

    l2_dirn = '/css/viirs/data/Level2/VNP47MCBEF/2019/'
    f_fname = l2_dirn + 'VNP47MCBEF.Stats.A2019%03d.0000_2400.Uniform_v_1_0_0.nc'%day
    m_fname = l2_dirn + 'VNP47MCBEF.Met.A2019%03d.nc'%day
    
    f = xr.open_dataset(f_fname) # fire property statistics
    m = xr.open_dataset(m_fname) # sample met fields
    
    if Verbose:
        display(Markdown("**FIRE PROPERTIES**"),f)
        display(Markdown("**SAMPLED METEOROLOGY**"),m)
    
    return f, m


In [3]:
pr.omp_set_num_threads(1) # number of OMP threads
p_dirn = './'
for day in [182,]:
    
    # Load fires and met fields
    # -------------------------
    f, m = getFiresMetOneDay(day,Verbose=True)
    
    # Calculate plume rise
    # --------------------
    z_i, z_d, z_a, z_f, rc = pr.getPlumesVMD(f.HeatFlux_f, f.Area_f, m,Verb=True) 
    
    # Calculate vertical mass distribution
    # ------------------------------------
    ds = pr.getVMD(z_i, z_d, z_a, z_f, m)
    
    # Save plume rise results to a file
    # ---------------------------------
    ds.to_netcdf(p_dirn+'VNP47MCBEF.PR.A2019%03d.nc'%day)
    
    display(ds)

 Open MP maximum number of threads:            1


**FIRE PROPERTIES**

<xarray.Dataset>
Dimensions:         (fire: 9365, stat: 5)
Coordinates:
    FP_Latitude     (fire) float32 ...
    FP_Longitude    (fire) float32 ...
    FP_Time         (fire) datetime64[ns] ...
Dimensions without coordinates: fire, stat
Data variables: (12/30)
    crs             int64 ...
    t_b             (fire) float32 ...
    C               (fire) float32 ...
    QA_flag         (fire) uint32 ...
    FP_Power_R_AC   (fire) float32 ...
    FP_confidence   (fire) int8 ...
    ...              ...
    t_f             (fire, stat) float32 ...
    t_s             (fire, stat) float32 ...
    p_s             (fire, stat) float32 ...
    p_f             (fire, stat) float32 ...
    Area_s          (fire, stat) float32 ...
    Area_f          (fire, stat) float32 ...
Attributes: (12/40)
    history:             Created Sat Nov 16 14:07:44 2024 main_reprocessing.py
    LongName:            MCBEF fire parameter estimation on FILDA VNP47MOD.
    satellite:           VNP
    sel_bg_bands:        ['M14', 'M15', 'I05', 'M16']
    sel_fire_bands:      ['DNB', 'M11', 'M13', 'M14', 'M15', 'M16']
    num_draw:            500
    ...                  ...
    sigma_C:             0.5
    thd_frp:             0.0
    frp_sigma_scale:     0.2
    SNR_bg:              {'DNB': 6.0, 'M08': 20.0, 'M10': 20.0, 'M11': 30.0, ...
    SNR_fire:            {'DNB': 6.0, 'M08': 20.0, 'M10': 40.0, 'M11': 60.0, ...
    precompile_string:   ,compiledir_mode=readonly

**SAMPLED METEOROLOGY**

<xarray.Dataset>
Dimensions:  (time: 9365, lev: 72)
Coordinates:
    lon      (time) float32 ...
    lat      (time) float32 ...
  * time     (time) datetime64[ns] 2019-07-01 2019-07-01 ... 2019-07-01T23:54:00
  * lev      (lev) float64 1.0 2.0 3.0 4.0 5.0 6.0 ... 68.0 69.0 70.0 71.0 72.0
Data variables: (12/15)
    PBLH     (time) float32 ...
    USTAR    (time) float32 ...
    TSH      (time) float32 ...
    BSTAR    (time) float32 ...
    EFLUX    (time) float32 ...
    HFLUX    (time) float32 ...
    ...       ...
    T        (time, lev) float32 ...
    H        (time, lev) float32 ...
    QV       (time, lev) float32 ...
    DELP     (time, lev) float32 ...
    PS       (time) float32 ...
    PHIS     (time) float32 ...

 PlumesVMD: Open MP maximum number of threads:            1

                   Plume Rise Estimation
                   ----------------------

          |    Lon    Lat  |    z_i      z_f   |   z_a      z_d    |  rc
  index   |    deg    deg  |     km       km   |    km       km    | 
--------- |  ------ ------ | -------- -------- | -------- -------- | ---
       0  |   28.47  11.33 |    -0.00    -0.00 |    -0.00    -0.00 |   0
      93  |   15.24  -2.86 |    -0.00    -0.00 |    -0.00    -0.00 |   0
     186  |   27.16  -6.20 |    -0.00    -0.00 |    -0.00    -0.00 |   0
     279  |   19.76  -6.69 |    -0.00    -0.00 |    -0.00    -0.00 |   0
     372  |   20.21  -7.87 |    -0.00    -0.00 |    -0.00    -0.00 |   0
     465  |   20.71  -8.68 |    -0.00    -0.00 |    -0.00    -0.00 |   0
     558  |   21.04  -9.19 |    -0.00    -0.00 |    -0.00    -0.00 |   0
     651  |   23.28 -10.67 |    -0.00    -0.00 |    -0.00    -0.00 |   0
     744  |   26.07 -14.56 |    -0.00    -0.00 |    -0.

<xarray.Dataset>
Dimensions:  (time: 9365, lev: 42)
Coordinates:
  * time     (time) datetime64[ns] 2019-07-01 2019-07-01 ... 2019-07-01T23:54:00
  * lev      (lev) float64 31.0 32.0 33.0 34.0 35.0 ... 68.0 69.0 70.0 71.0 72.0
    lon      (time) float32 ...
    lat      (time) float32 ...
Data variables:
    z_i      (time) float32 -1.0 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 1.4e+03 -1.0
    z_d      (time) float32 -1.0 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 1.7e+03 -1.0
    z_a      (time) float32 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 1.802e+03 -1.0
    z_f      (time) float32 -1.0 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 2.2e+03 -1.0
    z_c      (time) float32 -1.0 -1.0 -1.0 -1.0 -1.0 ... -1.0 -1.0 1.8e+03 -1.0
    delta    (time) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 400.0 0.0
    vmd      (time, lev) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    z        (time, lev) float32 2.318e+04 2.207e+04 2.101e+04 ... 191.1 63.45
    zs       (time) float32 504.4 503.6 482.5 482.9 ... 493.1 574.8 322.9 150.4
Attributes:
    option:   centered